### Partes de un tensor

La ventaja que tiene sobre NumPy es que es acelerado por GPU y **autograd**.

Los tensores de imágenes se leen así: (C, H, W)

C: Número de canales.

H: Altura de la imagen.

W: Anchura de la imagen.

In [1]:
import torch
from torchvision import datasets 
from torchvision.transforms import ToTensor # Convierte una imagen a tensor
import matplotlib.pyplot as plt


In [4]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.9.0+cu126
True
Tesla T4


In [5]:
# training_data y test_data son objetos de Dataset

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor() # Se aplica a cada imagen cuando se accede, no a todas de una
)

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.17MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 131kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 13.4MB/s]


In [6]:
training_data[0][0].size()

torch.Size([1, 28, 28])

In [7]:
import numpy as np
from collections import Counter # Estructura de datos para contar frecuencias de elementos

labels = [training_data[i][1] for i in range(len(training_data))]
counts = Counter(labels)

print(counts)

Counter({1: 6742, 7: 6265, 3: 6131, 2: 5958, 9: 5949, 0: 5923, 6: 5918, 8: 5851, 4: 5842, 5: 5421})


DataLoader: es un iterador inteligente de PyTorch. Toma un Dataset y lo convierte en lotes (batches) 

iter(train_loader): convierte el DataLoader en un iterador. Es como decir for batch in train_loader

In [ ]:
# DATALOADER 
from torch.utils.data import DataLoader

batch_size = 64 # Cuántos ejemplos se procesan juntos en una sola pasada por el modelo
                # El modelo "ve" 64 imágenes al mismo tiempo
                

train_loader = DataLoader(
    training_data,
    batch_size=batch_size,
    shuffle=True # Mezcla los datos en cada época
)

test_loader = DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False # Para que los resultados sean reproducibles
)

X, y = next(iter(train_loader)) 
print(X.shape) # (64, 1, 28, 28) 
print(y.shape) # (64,)


torch.Size([64, 1, 28, 28])
torch.Size([64])


64

In [ ]:
import torch.nn as nn # Define piezas estructurales de la red que tienen parámetros entrenables (pesos y sesgos)
import torch.nn.functional as F # Aplica funciones sobre los datos que no tienen parámetros propios. Son acciones matemáticas que no necesitan recordad nada de la iteración anterior

class SimpleCNN(nn.Module): # nn.Module es la madre de todas las redes
    def __init__(self):
        super().__init__() # Conecta la clase con todas las funciones internas de PyTorch
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # (N,16,14,14)
        x = self.pool(F.relu(self.conv2(x)))  # (N,32,7,7)
        x = x.view(x.size(0), -1)              # flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0

    for X, y in loader:
        X = X.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


In [ ]:
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in loader:
            X = X.to(device)
            y = y.to(device)

            outputs = model(X)
            _, predicted = torch.max(outputs, 1)

            total += y.size(0)
            correct += (predicted == y).sum().item()

    return correct / total


In [ ]:
class Robot:
    def __init__(self, nombre):
        self.nombre = nombre
        
    def saludar(self):
        print(f"Hola, soy {self.nombre}")
        
    def dinero(self, din):
        self.din = din
        
    def cant(self):
        print(self.din)
        
rob = Robot("ola")
rob.nombre


AttributeError: 'Robot' object has no attribute 'din'